In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
(x_train,y_train), (x_test,y_test) = mnist.load_data()
x_train = x_train/255
x_test = x_test/255
x_train = x_train.reshape((x_train.shape[0],x_train.shape[1],x_train.shape[2],1))
x_test = x_test.reshape((x_test.shape[0],x_test.shape[1],x_test.shape[2],1))
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(60000, 28, 28, 1)
(60000,)
(10000, 28, 28, 1)
(10000,)


In [2]:
import numpy as np
labels = np.zeros((60000,10))
for i in range(len(y_train)):
    labels[i,y_train[i]] = 1
print(labels.shape)

(60000, 10)


In [3]:
from tensorflow.keras.models import *
from tensorflow.keras.layers import *

In [4]:
model = Sequential()
model.add(Conv2D(64, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(28,28,1)))
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(Flatten())
model.add(Dense(10, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 64)        640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 32)        18464     
_________________________________________________________________
flatten (Flatten)            (None, 18432)             0         
_________________________________________________________________
dense (Dense)                (None, 10)                184330    
Total params: 203,434
Trainable params: 203,434
Non-trainable params: 0
_________________________________________________________________


In [5]:
generator = Sequential()

In [6]:
mu, sigma = 0, 1 # mean and standard deviation
s = np.random.normal(mu, sigma, 1000)
print(s.shape)

(1000,)


In [7]:
loss_object = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()
def cal_loss(model,x,y):
    y_pr = model(x)
    loss = loss_object(y_true=y,y_pred = y_pr)
    return loss,y_pr

def update_model_gradients(model,inputs,targets,learning_rate=0.001):
    with tf.GradientTape() as tape:
        loss_value,y_pr = cal_loss(model,inputs,targets)
    gradients = tape.gradient(loss_value,model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        
    return loss,y_pr
    

In [8]:
# Train on 10 training examples
accuracies = []
loss = []

num_epochs = 10
for epoch in range(num_epochs):
    print('Epoch {}'.format(epoch))
    epoch_acc = tf.keras.metrics.CategoricalAccuracy()
    loss,y_pr = update_model_gradients(model,x_train[:10],labels[:10])
    acc = epoch_acc(labels[:10],y_pr)
    print(acc.numpy())
    accuracies.append(acc)
    

W1015 23:15:39.726161 13536 base_layer.py:1814] Layer conv2d is casting an input tensor from dtype float64 to the layer's dtype of float32, which is new behavior in TensorFlow 2.  The layer has dtype float32 because it's dtype defaults to floatx.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



Epoch 0
0.0
Epoch 1
1.0
Epoch 2
1.0
Epoch 3
1.0
Epoch 4
1.0
Epoch 5
1.0
Epoch 6
1.0
Epoch 7
1.0
Epoch 8
1.0
Epoch 9
1.0


In [9]:
#Train on 60'000 training examples
accuracies = []
loss = []

num_epochs = 10
for epoch in range(num_epochs):
    print('Epoch {}'.format(epoch))
    epoch_acc = tf.keras.metrics.CategoricalAccuracy()
    loss,y_pr = update_model_gradients(model,x_train,labels)
    acc = epoch_acc(labels,y_pr)
    print(acc.numpy())
    accuracies.append(acc)

Epoch 0


ResourceExhaustedError: OOM when allocating tensor with shape[60000,26,26,64] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Conv2D]